In [2]:
import regionmask
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import json
import glob

from climaf.api import *

pattern = '/data/jservon/IPCC/wind/CORDEX_individual_models/CORDEX-${CORDEX_domain}_${experiment}_${variable}_${period}_${member}.nc'
cproject('wind_individual_models_cordex_ch12','CORDEX_domain','experiment','period','member',('variable','wind'), ensemble=['member'], separator='%')
dataloc(project='wind_individual_models_cordex_ch12', url=pattern)    

exp_dict_list = dict(
    baseline = dict(experiment='historical',
         period='1995-2005'
        ),
    baseline_ext = dict(experiment='rcp85',
         period='2006-2014'
        ),
    rcp26_mid = dict(experiment='rcp26',
         period='2041-2060'
        ),
    rcp26_far = dict(experiment='rcp26',
         period='2081-2100'
        ),
    rcp85_mid = dict(experiment='rcp85',
         period='2041-2060'
        ),
    rcp85_far = dict(experiment='rcp85',
         period='2081-2100'
        )  
)


AR6regions_by_CORDEX_domain = dict(
    AUS = ['NAU','CAU','EAU','SAU','NZ'],
    SEA = ['SEA'],
    WAS = ['ARP','SAS','WCA'],#,'TIB'],
    EAS = ['TIB','ECA','EAS'],#'SAS'
    CAM = ['NSA','SCA','CAR'],
    SAM = ['NWS','NSA','SAM','NES','SWS','SES','SSA'],
    NAM = ['NWN','NEN','WNA','CNA','ENA','NCA'],#,'GAP'],
    #EUR = ['MED','WCE','NEU'],
    AFR = ['WAF','SAH','CAF','WSAF','ESAF','MDG','SEAF','NEAF','ARP']
)


CORDEX_domains = AR6regions_by_CORDEX_domain.keys()

all_CORDEX_domains_ens_exp_dict = dict()

for CORDEX_domain in CORDEX_domains:

    ens_exp_dict = dict()
    lom_per_exp = dict()
    for exp in exp_dict_list:
        print exp

        experiment = exp_dict_list[exp]['experiment']
        period = exp_dict_list[exp]['period']

        req = ds(experiment=experiment,
                 CORDEX_domain = CORDEX_domain,
                 period=period,
                 member='*',
                 project='wind_individual_models_cordex_ch12')
        try:
            ens_exp = req.explore('ensemble')
            #
            # -- Climatologies
            clim_exp      = clim_average(ens_exp, 'ANM')

            lom_per_exp[exp] = clim_exp.keys()

            # -- Changes = Scenario minus baselines
            ens_exp_dict[exp] = clim_exp
        except:
            lom_per_exp[exp] = []
            # -- Changes = Scenario minus baselines
            ens_exp_dict[exp] = dict()
            

    lom_baseline     = lom_per_exp['baseline']
    lom_baseline_ext = lom_per_exp['baseline_ext']
    print 'Models not in both sets:'
    print set(lom_baseline) ^ set(lom_baseline_ext)
    print 'Models in common:'
    common_lom_baseline = list( set(lom_baseline) & set(lom_baseline_ext) )
    print common_lom_baseline



    req_dict = dict(project='wind_individual_models_cordex_ch12')
    wreq_dict = req_dict.copy()
    wreq_dict.update(exp_dict_list['baseline'])
    ens_baseline_hist = eds(member=common_lom_baseline,
                            CORDEX_domain = CORDEX_domain,
                            **wreq_dict
                           )

    wreq_dict = req_dict.copy()
    wreq_dict.update(exp_dict_list['baseline_ext'])
    ens_baseline_ext = eds(member=common_lom_baseline,
                           CORDEX_domain = CORDEX_domain,
                           **wreq_dict
                          )
    ens_baseline_dict = dict()
    for mem in ens_baseline_hist:
        ens_baseline_dict[mem] = ccdo2(ens_baseline_hist[mem], ens_baseline_ext[mem], operator='cat')

    # -- Add to the list of ensembles
    ens_baseline = cens(ens_baseline_dict)
    ens_exp_dict['baseline'] = clim_average(ens_baseline, 'ANM')

    ens_exp_dict.pop('baseline_ext')
    
    all_CORDEX_domains_ens_exp_dict[CORDEX_domain] = ens_exp_dict


import csv

GWL_csv = '/home/jservon/Chapter12_IPCC/scripts/ATLAS/warming-levels/CMIP5_Atlas_WarmingLevels.csv'

GWL_dict = dict()
i = 0
with open(GWL_csv) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')#, quotechar='|')
    for row in spamreader:
        print row
        model = row[0].split('_')[0]
        print model
        GWL_dict[model] = dict()
        if i==0:
            colnames = row
        j = 0
        for elt in row:
            print elt
            GWL_dict[model][colnames[j]] = row[j]
            j = j + 1
        i = i + 1
        
#

all_CORDEX_domains_ens_dict_per_GWL = dict()

for CORDEX_domain in CORDEX_domains:

    ens_dict_per_GWL = dict()
    list_of_GWLs = ['1.5','2','4']

    for GWL in list_of_GWLs:
        ens_dict_per_GWL[GWL] = dict()

    req_dict = dict(project='wind_individual_models_cordex_ch12')

    for scenario in ['26','85']:
        list_of_models = all_CORDEX_domains_ens_exp_dict[CORDEX_domain]['rcp'+scenario+'_far'].keys()
        for wmodel_realization in list_of_models:
            wmodel = wmodel_realization.split('_')[0]
            GWL_model = None
            for tmp_GWL_model in GWL_dict:
                if tmp_GWL_model in wmodel:
                    GWL_model = tmp_GWL_model
                    break
                    
            if GWL_model:
                print 'We have : ', wmodel
                print GWL_dict[GWL_model]
                for GWL in list_of_GWLs:
                    if scenario=='26': GWL_scenario = GWL+'_rcp26'
                    if scenario=='85': GWL_scenario = GWL+'_rcp85'

                    # --> file nc
                    # --> period
                    central_year = GWL_dict[GWL_model][GWL_scenario]
                    if central_year not in ['NA','9999'] and float(central_year)>=2024:
                        start_year = str( int(central_year)-9 )
                        end_year = str( int(central_year)+10 )

                        dat = ds(member = wmodel_realization,
                                 CORDEX_domain = CORDEX_domain,
                                 experiment = 'rcp'+scenario,
                                 period=start_year+'-'+end_year,
                                 **req_dict
                                 )
                        ens_dict_per_GWL[GWL][wmodel_realization+'_'+scenario] = clim_average(dat, 'ANM')
                        print cfile(ens_dict_per_GWL[GWL][wmodel_realization+'_'+scenario])
            else:
                print 'We dont have GWL info for ',wmodel
    #    
    all_CORDEX_domains_ens_dict_per_GWL[CORDEX_domain] = ens_dict_per_GWL

baseline_ext
rcp85_far
rcp26_far
baseline
rcp26_mid
rcp85_mid
Models not in both sets:
set([])
Models in common:
['NOAA-GFDL-GFDL-ESM2M_r1i1p1_SMHI-RCA4_v1', 'CCCma-CanESM2_r1i1p1_SMHI-RCA4_v1', 'ICHEC-EC-EARTH_r12i1p1_SMHI-RCA4_v1', 'MOHC-HadGEM2-ES_r1i1p1_ICTP-RegCM4-3_v4', 'MOHC-HadGEM2-ES_r1i1p1_GERICS-REMO2015_v1', 'MPI-M-MPI-ESM-LR_r1i1p1_SMHI-RCA4_v1', 'CSIRO-QCCCE-CSIRO-Mk3-6-0_r1i1p1_SMHI-RCA4_v1', 'MPI-M-MPI-ESM-MR_r1i1p1_ICTP-RegCM4-3_v4', 'NOAA-GFDL-GFDL-ESM2M_r1i1p1_ICTP-RegCM4-7_v0', 'NCC-NorESM1-M_r1i1p1_SMHI-RCA4_v1', 'MIROC-MIROC5_r1i1p1_SMHI-RCA4_v1', 'CNRM-CERFACS-CNRM-CM5_r1i1p1_SMHI-RCA4_v1', 'MOHC-HadGEM2-ES_r1i1p1_SMHI-RCA4_v1', 'MPI-M-MPI-ESM-MR_r1i1p1_ICTP-RegCM4-7_v0', 'MPI-M-MPI-ESM-LR_r1i1p1_GERICS-REMO2015_v1', 'NCC-NorESM1-M_r1i1p1_GERICS-REMO2015_v1', 'IPSL-IPSL-CM5A-MR_r1i1p1_SMHI-RCA4_v1', 'MOHC-HadGEM2-ES_r1i1p1_ICTP-RegCM4-7_v0']
baseline_ext
rcp85_far
rcp26_far
baseline
rcp26_mid
rcp85_mid
Models not in both sets:
set(['NOAA-GFDL-GFDL-ESM2M_r1i1p1_IC

/data/jservon/climafcache/ef/36854a61e51184510293acc318bd5a6cb0ea19305397732469e407.nc
We have :  NCC-NorESM1-M
{'4_rcp45': 'NA', '1.5_rcp26': 'NA', '2_rcp85': '2048', '2_rcp45': '2072', '1.5_rcp85': '2032', '3_rcp85': '2072', '4_rcp85': 'NA', '3_rcp45': 'NA', '4_rcp26': 'NA', '1.5_rcp45': '2039', '3_rcp26': 'NA', 'model_run': 'NorESM1-M_r1i1p1', '2_rcp26': 'NA'}
/data/jservon/climafcache/0b/ed19739adfdb421ef53f3850ba7bdd10c3e210a2e78ed81de24e61.nc
/data/jservon/climafcache/90/1638534d13f52c952f4da2e6b32639d23e8ab717a32019948106f0.nc
We have :  MPI-M-MPI-ESM-LR
{'4_rcp45': 'NA', '1.5_rcp26': '2022', '2_rcp85': '2036', '2_rcp45': '2042', '1.5_rcp85': '2017', '3_rcp85': '2061', '4_rcp85': '2081', '3_rcp45': 'NA', '4_rcp26': 'NA', '1.5_rcp45': '2022', '3_rcp26': 'NA', 'model_run': 'MPI-ESM-LR_r1i1p1', '2_rcp26': 'NA'}
We have :  MOHC-HadGEM2-ES
{'4_rcp45': 'NA', '1.5_rcp26': '2023', '2_rcp85': '2035', '2_rcp45': '2043', '1.5_rcp85': '2023', '3_rcp85': '2054', '4_rcp85': '2071', '3_rcp45':

/data/jservon/climafcache/d5/13167a8b8b44389f737e3864469d3844ae9117f57c40836d2d1df8.nc
We have :  NCC-NorESM1-M
{'4_rcp45': 'NA', '1.5_rcp26': 'NA', '2_rcp85': '2048', '2_rcp45': '2072', '1.5_rcp85': '2032', '3_rcp85': '2072', '4_rcp85': 'NA', '3_rcp45': 'NA', '4_rcp26': 'NA', '1.5_rcp45': '2039', '3_rcp26': 'NA', 'model_run': 'NorESM1-M_r1i1p1', '2_rcp26': 'NA'}
/data/jservon/climafcache/3e/bc5fad1b1b2786cb6de00958d6077b89b9ac0afd03f1a30529d271.nc
/data/jservon/climafcache/b7/0ef9785a5e7f0fac9646b2ef8129a3f035d97ab096200c3dfae674.nc
We have :  MPI-M-MPI-ESM-MR
{'4_rcp45': 'NA', '1.5_rcp26': '2023', '2_rcp85': '2038', '2_rcp45': '2045', '1.5_rcp85': '2019', '3_rcp85': '2060', '4_rcp85': '2082', '3_rcp45': 'NA', '4_rcp26': 'NA', '1.5_rcp45': '2023', '3_rcp26': 'NA', 'model_run': 'MPI-ESM-MR_r1i1p1', '2_rcp26': 'NA'}
/data/jservon/climafcache/72/e35b59f085b8d36c90812c3e2192a07bed04ec4919b4556440457d.nc
/data/jservon/climafcache/21/1107aacf9bedd9c86be87c3ecebb103463aa68aba6a5f182f8fee2.nc

/data/jservon/climafcache/1c/b2e20f1a46ed330835b7403f7f0b74ba3d09e33fe5519848f8f701.nc
We have :  MIROC-MIROC5
{'4_rcp45': 'NA', '1.5_rcp26': '2046', '2_rcp85': '2048', '2_rcp45': '2071', '1.5_rcp85': '2033', '3_rcp85': '2071', '4_rcp85': 'NA', '3_rcp45': 'NA', '4_rcp26': 'NA', '1.5_rcp45': '2039', '3_rcp26': 'NA', 'model_run': 'MIROC5_r1i1p1', '2_rcp26': 'NA'}
/data/jservon/climafcache/ca/70efd4b9b59c37b0a058d39c21f263d2b61135dae6395d99f1b7df.nc
/data/jservon/climafcache/f4/8d7b8c9dbcfa7dc870f108927d30261fdad78857e478e03bf216d5.nc
We have :  MIROC-MIROC5
{'4_rcp45': 'NA', '1.5_rcp26': '2046', '2_rcp85': '2048', '2_rcp45': '2071', '1.5_rcp85': '2033', '3_rcp85': '2071', '4_rcp85': 'NA', '3_rcp45': 'NA', '4_rcp26': 'NA', '1.5_rcp45': '2039', '3_rcp26': 'NA', 'model_run': 'MIROC5_r1i1p1', '2_rcp26': 'NA'}
/data/jservon/climafcache/ff/a38bcf4ebdfdfc63dfa5e81e2b040a377b01904e1d2855d17487cd.nc
/data/jservon/climafcache/e5/c430e911df52880ce4615d0abf3e3ff79b06652a326c5643017a11.nc
We have :  

In [3]:
all_CORDEX_domains_ens_dict_per_GWL['SAM'].keys()

['1.5', '2', '4']

In [3]:
clog('critical')
pattern = '/data/jservon/IPCC/WSPD/${experiment}/sfcWind.${member}.${clim_period}.nc'
cproject('wind_individual_models_EURO_cordex_ch12',('CORDEX_domain','EUR'),'experiment',('period','fx'),'clim_period','member',('variable','sfcWind'), ensemble=['member'], separator='%')
dataloc(project='wind_individual_models_EURO_cordex_ch12', url=pattern)    

<climaf.dataloc.dataloc instance at 0x2b0a09aed4d0>

In [4]:
all_CORDEX_domains_ens_dict_per_GWL['EUR'] = dict()

for gwl in ['1.5', '2', '4']:
    
    req = ds(project = 'wind_individual_models_EURO_cordex_ch12',
              clim_period = 'gwl'+gwl.replace('.',''),
              member = '*',
              experiment = '*'
             )
    all_CORDEX_domains_ens_dict_per_GWL['EUR'][gwl] = req.explore('ensemble')

In [10]:
all_CORDEX_domains_ens_exp_dict['SAM'].keys()

['rcp85_mid', 'rcp26_mid', 'rcp85_far', 'rcp26_far', 'baseline']

In [5]:
all_CORDEX_domains_ens_exp_dict['EUR'] = dict()
for clim_period in ['rcp85_mid', 'rcp26_mid', 'rcp85_far', 'rcp26_far', 'baseline']:
    if clim_period=='baseline':
        wclim_period = 'ref'
        experiment = 'RCP85'
    if '_' in clim_period:
        experiment = clim_period.split('_')[0].upper()
    if 'far' in clim_period:
        wclim_period = 'ece'
    if 'mid' in clim_period:
        wclim_period = 'mce'
    req = ds(project = 'wind_individual_models_EURO_cordex_ch12',
              clim_period = wclim_period,
              member = '*',
              experiment = experiment
             )
    all_CORDEX_domains_ens_exp_dict['EUR'][clim_period] = req.explore('ensemble')    


In [20]:
def weighted_mean(da, weights, dim):
    """Reduce da by a weighted mean along some dimension(s).

    Parameters
    ----------
    da : DataArray
        Object over which the weighted reduction operation is applied.    
    weights : DataArray
        An array of weights associated with the values in this Dataset.
    dim : str or sequence of str, optional
        Dimension(s) over which to apply the weighted `mean`.
        
    Returns
    -------
    weighted_mean : DataArray
        New DataArray with weighted mean applied to its data and
        the indicated dimension(s) removed.
    """

    weighted_sum = (da * weights).sum(dim=dim, skipna=True)
    # need to mask weights where data is not valid
    masked_weights = weights.where(da.notnull())
    sum_of_weights = masked_weights.sum(dim=dim, skipna=True)
    valid_weights = sum_of_weights != 0
    sum_of_weights = sum_of_weights.where(valid_weights)

    return weighted_sum / sum_of_weights


#
def average_over_AR6_region(filename, variable, region_name):

    # -- AR6 regions
    #ar6_all = regionmask.defined_regions.ar6.all
    # -- Get the regions
    ar6_land = regionmask.defined_regions.ar6.land

    #ax = ar6_all.plot()
    # -- Get land/sea mask (generic)
    land_110 = regionmask.defined_regions.natural_earth.land_110

    # -- Get data
    ds = xr.open_dataset(filename, decode_times=False)
    dat = ds[variable]
    dat.values = np.array(dat.values, dtype=np.float32)

    # -- Mask the data
    mask_3D = ar6_land.mask_3D(dat) # AR6 mask
    land_mask = land_110.mask_3D(dat) # Land sea mask
    mask_lsm = mask_3D * land_mask.squeeze(drop=True) # Combine the two
    
    # -- Compute weights
    if dat.lat.shape == dat.shape or region_name=='all':
        weights = np.cos(np.deg2rad(dat.lat))
        tmp = weighted_mean(dat, mask_lsm * weights, ("lon", "lat"))
        tmp['abbrevs'] = mask_3D.abbrevs
        return tmp
    else:
        # -- Case dat is has time dim
        if 'time' in dat.dims:
            matlat = np.mean(dat.values, axis=dat.dims.index('time')) * 0
        else:
            matlat = dat.values * 0

        if dat.dims.index('lat')<dat.dims.index('lon'):
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[:,i] = dat.lat
        else:
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[i,:] = dat.lat
    
        weights = np.cos(np.deg2rad(matlat))
    
        if isinstance(region_name, list):
            res = list()
            for region in region_name:
                region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region))
                dat_region = dat.where(region_mask)
                weights_region = np.where(region_mask, weights, float("nan"))
                #weights_region = weights.where(region_mask)
                res.append( weighted_mean(dat_region, region_mask*weights_region, ("lon","lat")) )
            return res
        else:
            region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region_name))
            dat_region = dat.where(region_mask)
            weights_region = np.where(region_mask, weights, float("nan"))
            #weights_region = weights.where(region_mask)            
            return weighted_mean(dat_region, region_mask*weights_region, ("lon","lat"))
#


def regions_contained(lon, lat, regions):

    # determine if the longitude needs to be wrapped
    regions_is_180 = regions.lon_180
    grid_is_180 = regionmask.core.utils._is_180(lon.min(), lon.max())

    wrap_lon = not regions_is_180 == grid_is_180

    lon_orig = lon.copy()
    if wrap_lon:
        lon = regionmask.core.utils._wrapAngle(lon, wrap_lon)

    lon = np.asarray(lon).squeeze()
    lat = np.asarray(lat).squeeze()

    if lon.ndim == 1 and lat.ndim == 1:
        poly = shapely.geometry.box(lon.min(), lat.min(), lon.max(), lat.max())

    # convex_hull is not really what we need
    # https://gist.github.com/dwyerk/10561690
    #     elif lon.ndim == 2 and lat.ndim == 2:
    #         # get the convex hull from all points
    #         lonlat = np.stack([lon.ravel(), lat.ravel()], axis=1)
    #         multipoint = shapely.geometry.MultiPoint(lonlat)
    #         poly = multipoint.convex_hull
    else:
        raise ValueError("Cannot currently handle 2D coordinates")

    fully_contained = list()
    for region_poly in regions.polygons:
        res = poly.contains(region_poly)

        fully_contained.append(res)

    return xr.DataArray(
        fully_contained, dims=["region"], coords=dict(region=regions.numbers)
    )

#

AR6regions_by_CORDEX_domain = dict(
    AUS = ['NAU','CAU','EAU','SAU','NZ'],
    SEA = ['SEA'],
    WAS = ['ARP','SAS','WCA'],#,'TIB'],
    EAS = ['TIB','ECA','EAS'],#'SAS'
    CAM = ['NSA','SCA','CAR'],
    SAM = ['NWS','NSA','SAM','NES','SWS','SES','SSA'],
    NAM = ['NWN','NEN','WNA','CNA','ENA','NCA'],#,'GAP'],
    EUR = ['MED','WCE','NEU'],
    AFR = ['WAF','SAH','CAF','WSAF','ESAF','MDG','SEAF','NEAF','ARP']
)



regional_averages_diff = dict()
#for CORDEX_domain in all_CORDEX_domains_ens_exp_dict.keys():
for CORDEX_domain in ['EUR']:
    
    short_CORDEX_domain = CORDEX_domain[0:3]
    
    for ens_CORDEX in [all_CORDEX_domains_ens_exp_dict[CORDEX_domain],
                       all_CORDEX_domains_ens_dict_per_GWL[CORDEX_domain]]:
                
        # -- Loop on experiments / horizons
        for clim_period in ens_CORDEX:
            if 'baseline' not in clim_period:
                print clim_period
                if clim_period not in regional_averages_diff:
                    regional_averages_diff[clim_period] = dict()

                # -- Loop on the members of each ensemble
                for mem in ens_CORDEX[clim_period]:
                    wmem = mem.replace('_85','').replace('_26','')
                    if wmem in all_CORDEX_domains_ens_exp_dict[CORDEX_domain]['baseline']:
                        print wmem
                        if short_CORDEX_domain=='EUR':
                            variable='sfcWind'
                        else:
                            variable='wind'
                        # -- Compute the averages for each AR6 region thanks to regionmask
                        # --> We regrid on a 0.25° grid with conservative regridding
                        tmp = average_over_AR6_region(
                                    cfile(regridn(ens_CORDEX[clim_period][mem], cdogrid='r1440x720', option='remapbil')),
                                    variable, 'all')
                        tmp_baseline = average_over_AR6_region(
                                    cfile(regridn(all_CORDEX_domains_ens_exp_dict[CORDEX_domain]['baseline'][wmem], cdogrid='r1440x720', option='remapbil')),
                                    variable, 'all')
                        region_names = tmp.abbrevs
                        for region_name in AR6regions_by_CORDEX_domain[short_CORDEX_domain]:
                            print region_name
                            region_value = float(tmp.sel(region=list(tmp.abbrevs).index(region_name))[0].values)
                            region_value_baseline = float(tmp_baseline.sel(region=list(tmp.abbrevs).index(region_name))[0].values)
                            print region_value, region_value_baseline
                            if region_value_baseline==0:
                                if region_value==0:
                                    perc_val = 0
                                else:
                                    perc_val = float('nan')
                            else:
                                perc_val = 100*(region_value - region_value_baseline)/region_value_baseline
                            if region_name not in regional_averages_diff[clim_period]:
                                regional_averages_diff[clim_period][region_name] = [perc_val]
                            else:
                                regional_averages_diff[clim_period][region_name].append(perc_val)

                        
                        
quantiles_dict = dict()
for clim_period in regional_averages_diff:
    quantiles_dict[clim_period] = dict()
    for region_name in regional_averages_diff[clim_period]:
        print clim_period, region_name
        quantiles_dict[clim_period][region_name] = dict()
        dat = np.array(regional_averages_diff[clim_period][region_name])
        wdat = dat[~np.isnan(dat)]
        if len(wdat)>=5:
            q10 = np.quantile(wdat, 0.1)
            q50 = np.quantile(wdat, 0.5)
            q90 = np.quantile(wdat, 0.9)
            quantiles_dict[clim_period][region_name] = [q10, q50, q90]
        else:
            quantiles_dict[clim_period][region_name] = [-99999, -99999, -99999]

import json

ensemble = 'CORDEX'
outfilename = '/home/jservon/Chapter12_IPCC/data/Figure_S12.5/'+ensemble+'_sfcWind_diff-perc-baseline_AR6_regional_averages.json'
#print outfilename
with open(outfilename, 'w') as fp:
    json.dump(quantiles_dict, fp, sort_keys=True, indent=4)

rcp26_mid
MPIr1-RACMO
MED
3.62760676782 3.61852800044
WCE
4.11452526818 4.10460983641
NEU
4.12365569088 4.12191252509
HADGEMr1-RCA
MED
3.93340491895 3.95581162214
WCE
3.84699832484 3.87531669714
NEU
4.3436518368 4.4421581052
CNRMr1-REMO
MED
3.72549229862 3.84157847894
WCE
3.54061904719 3.59161973291
NEU
3.68606249522 3.64245493012
NORESMr1-RACMO
MED
3.43278362569 3.40966284007
WCE
4.07075242927 4.05596170062
NEU
4.06787523246 4.06025079139
ECEARTHr12-CCLM
MED
3.2679138547 3.29453849571
WCE
3.59717933928 3.63361976694
NEU
3.67786678672 3.67933321567
ECEARTHr3-HIRHAM
MED
4.21630662081 4.22374606584
WCE
3.87845906088 3.92848884329
NEU
4.2000834758 4.21818138216
HADGEMr1-HIRHAM
MED
4.13838559499 4.18521473747
WCE
3.82418945834 3.86550413742
NEU
4.17325807625 4.22424298213
CNRMr1-ALADIN63
MED
3.30713457182 3.29240233884
WCE
3.53644596739 3.5111088047
NEU
3.23799563671 3.20521128231
NORESMr1-RCA
MED
3.78456512474 3.79261447194
WCE
3.87895900452 3.90151942751
NEU
4.28850075059 4.32015279781
M

Climaf_Driver_Error: "Something went wrong when computing regridn(ds('wind_individual_models_EURO_cordex_ch12%%sfcWind%fx%global%EUR%RCP85%ref%HADGEMr1-REGCM'),cdogrid='r1440x720',option='remapbil'). See file ./last.out for details"

In [10]:
CORDEX_domain = 'EUR'
errors_list = []
crm(pattern='HADGEMr1-REGCM')
for clim_period in all_CORDEX_domains_ens_exp_dict[CORDEX_domain]:
    for mem in all_CORDEX_domains_ens_exp_dict[CORDEX_domain][clim_period]:
        try:
            cfile(regridn(all_CORDEX_domains_ens_exp_dict[CORDEX_domain][clim_period][mem], cdogrid='r1440x720', option='remapbil'))
            print 'ok for ',mem, clim_period
        except:
            print 'Check',mem, clim_period
            errors_list.append(dict(mem=mem, clim_period=clim_period))
    

Removed files:
ok for  MPIr1-RACMO rcp26_mid
ok for  HADGEMr1-RCA rcp26_mid
ok for  CNRMr1-REMO rcp26_mid
ok for  NORESMr1-RACMO rcp26_mid
ok for  ECEARTHr12-CCLM rcp26_mid
ok for  ECEARTHr3-HIRHAM rcp26_mid
ok for  HADGEMr1-HIRHAM rcp26_mid
ok for  CNRMr1-ALADIN63 rcp26_mid
ok for  NORESMr1-RCA rcp26_mid
ok for  MIROCr1-REMO rcp26_mid
ok for  HADGEMr1-HADREM rcp26_mid
ok for  MPIr1-REMO rcp26_mid
ok for  HADGEMr1-REGCM rcp26_mid
ok for  MPIr1-CCLM rcp26_mid
ok for  NORESMr1-REMO rcp26_mid
ok for  MPIr2-REMO rcp26_mid
ok for  MPIr1-WRF361H rcp26_mid
ok for  MIROCr1-CCLM rcp26_mid
ok for  CNRMr1-RACMO rcp26_mid
ok for  HADGEMr1-RACMO rcp26_mid
ok for  MPIr1-REGCM rcp26_mid
ok for  HADGEMr1-REMO rcp26_mid
ok for  MPIr1-RCA rcp26_mid
ok for  ECEARTHr12-REMO rcp26_mid
ok for  ECEARTHr12-RCA rcp26_mid
ok for  ECEARTHr12-RACMO rcp26_mid
ok for  ECEARTHr12-HADREM rcp26_mid
ok for  MPIr1-RACMO rcp26_far
ok for  HADGEMr1-RCA rcp26_far
ok for  CNRMr1-REMO rcp26_far
ok for  NORESMr1-RACMO rcp26_f

In [29]:
import xarray as xr
def get_tracking_id(ncfile):
    nc = xr.open_dataset(ncfile)
    tracking_id = nc.tracking_id
    nc.close()
    return tracking_id
rows = [['DATA_REF_SYNTAX','MODEL_ID','RCM_VERSION_ID','FREQUENCY','VAR_NAME','VERSION_NO','HANDLE','SUBPANEL']]
CORDEX_domain = 'EUR'
short_CORDEX_domain = CORDEX_domain[0:3]
for exp in all_CORDEX_domains_ens_exp_dict[CORDEX_domain]:
    for mem in all_CORDEX_domains_ens_exp_dict[CORDEX_domain][exp]:
        try:
            dat = all_CORDEX_domains_ens_exp_dict[CORDEX_domain][exp][mem]
            print dat.baseFiles()
        except:
            dat = all_CORDEX_domains_ens_exp_dict[CORDEX_domain][exp][mem].operands[0]
            print dat.baseFiles()
        regions = 'NEU,WCE,MED'
        nc = xr.open_dataset(dat.baseFiles().split(' ')[0])
        institute_id = nc.institute_id
        driving_model_id = nc.driving_model_id
        frequency = nc.frequency
        model_version = nc.rcm_version_id
        realization = nc.driving_model_ensemble_member
        experiment = nc.driving_experiment_name
        try:
            tracking_id = nc.tracking_id
        except:
            tracking_id = 'none'
        rcm = nc.model_id
        variable = 'sfcWind'
        CORDEX_domain_id = nc.CORDEX_domain
        nc.close()
        #project_id.product.CORDEX_domain.institute_id.driving_model_id.driving_experiment_name.driving_model_ensemble_member
        dataset_descr = 'CORDEX.output.'+CORDEX_domain_id+'.'+institute_id+'.'+driving_model_id+'.'+experiment+'.'+realization+','+rcm+','+model_version+','+frequency+','+variable+','+'version_no'+','+tracking_id+','+regions
        print dataset_descr
        #ds_kvp = dat.explore('resolve').kvp
        #ncfile = dat.baseFiles().split(' ')[0]
        #regions = ','.join(AR6regions_by_CORDEX_domain[short_CORDEX_domain])
        #print regions
        #try:
        #    tracking_id = get_tracking_id(ncfile)
        #except:
        #    tracking_id = ''
        #dataset_descr = 'CORDEX.output.'+CORDEX_domain+'.'+ds_kvp['institute']+'.'+ds_kvp['driving_model']+'.'+ds_kvp['experiment']+'.'+ds_kvp['realization']+','+ds_kvp['model_version']+','+ds_kvp['frequency']+','+ds_kvp['variable']+','+'version_no'+','+tracking_id+','+regions
        #print(dataset_descr)
        #rows.append([dataset_descr])


/data/jservon/IPCC/WSPD/RCP26/sfcWind.MPIr1-RACMO.mce.nc
CORDEX.output.EUR-11.KNMI.MPI-M-MPI-ESM-LR.rcp26.r1i1p1,KNMI-RACMO22E,v1,day,sfcWind,version_no,7d6f0b9a-86c4-4a49-ab7d-72b3c235ac3b,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.HADGEMr1-RCA.mce.nc
CORDEX.output.EUR-11.SMHI.MOHC-HadGEM2-ES.rcp26.r1i1p1,SMHI-RCA4,v1,day,sfcWind,version_no,de246aa9-10e9-49c7-9982-3a79a928371d,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.CNRMr1-REMO.mce.nc
CORDEX.output.EUR-11.GERICS.CNRM-CERFACS-CNRM-CM5.rcp26.r1i1p1,GERICS-REMO2015,v2,day,sfcWind,version_no,hdl:21.14103/66ce107e-9097-4cc9-974c-ceae3747c5dc,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.NORESMr1-RACMO.mce.nc
CORDEX.output.EUR-11.KNMI.NCC-NorESM1-M.rcp26.r1i1p1,KNMI-RACMO22E,v1,day,sfcWind,version_no,ef878372-ec57-4428-9166-a819479a9b18,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.ECEARTHr12-CCLM.mce.nc
CORDEX.output.EUR-11.CLMcom.ICHEC-EC-EARTH.rcp26.r12i1p1,CLMcom-CCLM4-8-17,v1,day,sfcWind,version_no,none,NEU,WCE

CORDEX.output.EUR-11.CLMcom-BTU.MPI-M-MPI-ESM-LR.rcp26.r1i1p1,CLMcom-BTU-CCLM4-8-17,v1,day,sfcWind,version_no,87e09b9c-d93d-11e7-8b3a-b3ab53bf7722,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.NORESMr1-REMO.ece.nc
CORDEX.output.EUR-11.GERICS.NCC-NorESM1-M.rcp26.r1i1p1,GERICS-REMO2015,v1,day,sfcWind,version_no,hdl:21.14103/c928b32c-8bd7-42d7-a84c-ee01aa0f1940,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.MPIr2-REMO.ece.nc
CORDEX.output.EUR-11.MPI-CSC.MPI-M-MPI-ESM-LR.rcp26.r2i1p1,MPI-CSC-REMO2009,v1,day,sfcWind,version_no,3de6e68f-577e-46c9-8aaa-52a62441cacb,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.MPIr1-WRF361H.ece.nc
CORDEX.output.EUR-11.UHOH.MPI-M-MPI-ESM-LR.rcp26.r1i1p1,UHOH-WRF361H,v1,day,sfcWind,version_no,b23938e3-8743-45d2-ab4b-a36c3ad9a60c,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP26/sfcWind.MIROCr1-CCLM.ece.nc
CORDEX.output.EUR-11.CLMcom.MIROC-MIROC5.rcp26.r1i1p1,CLMcom-CCLM4-8-17,v1,day,sfcWind,version_no,788d266c-8c19-11e8-ad79-0cc47aab4a1c,NEU,WCE,MED
/data/js

CORDEX.output.EUR-11.CLMcom.MIROC-MIROC5.rcp85.r1i1p1,CLMcom-CCLM4-8-17,v1,day,sfcWind,version_no,2e040bf6-e084-11e7-b22c-81c28a935756,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.CNRMr1-RACMO.mce.nc
CORDEX.output.EUR-11.KNMI.CNRM-CERFACS-CNRM-CM5.rcp85.r1i1p1,KNMI-RACMO22E,v2,day,sfcWind,version_no,0334913c-21ed-49c2-98a3-3f47f0e7cbbf,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.HADGEMr1-RACMO.mce.nc
CORDEX.output.EUR-11.KNMI.MOHC-HadGEM2-ES.rcp85.r1i1p1,KNMI-RACMO22E,v2,day,sfcWind,version_no,a131c638-f1ba-404a-8785-e342c5bcc24e,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.MPIr1-REGCM.mce.nc
CORDEX.output.EUR-11.ICTP.MPI-M-MPI-ESM-LR.rcp85.r1i1p1,ICTP-RegCM4-6,v1,day,sfcWind,version_no,81ff1944-90a5-11e9-95a1-0894ef50228a,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.CANESMr1-REMO.mce.nc
CORDEX.output.EUR-11.GERICS.CCCma-CanESM2.rcp85.r1i1p1,GERICS-REMO2015,v1,day,sfcWind,version_no,44e67333-3e53-4dfd-abb4-929aa388e909,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcW

CORDEX.output.EUR-11.SMHI.MOHC-HadGEM2-ES.historical.r1i1p1,SMHI-RCA4,v1,day,sfcWind,version_no,c1157321-bd9f-4858-9865-628115d3440a,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.CANESMr1-CCLM.ref.nc
CORDEX.output.EUR-11.CLMcom.CCCma-CanESM2.historical.r1i1p1,CLMcom-CCLM4-8-17,v1,day,sfcWind,version_no,b84b1a3e-f56f-11e7-97ba-dd685a925ffb,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.HADGEMr1-HIRHAM.ref.nc
CORDEX.output.EUR-11.DMI.MOHC-HadGEM2-ES.historical.r1i1p1,DMI-HIRHAM5,v2,day,sfcWind,version_no,95d237b0-46b5-11e9-874e-bf126b2e63eb,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.ECEARTHr1-RCA.ref.nc
CORDEX.output.EUR-11.SMHI.ICHEC-EC-EARTH.historical.r1i1p1,SMHI-RCA4,v1,day,sfcWind,version_no,a58aafe7-06d9-409c-bac6-e0f2ece8e63c,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.HADGEMr1-HADREM.ref.nc
CORDEX.output.EUR-11.MOHC.MOHC-HadGEM2-ES.historical.r1i1p1,MOHC-HadREM3-GA7-05,v1,day,sfcWind,version_no,hdl:21.14100/c095ada1-e30f-4e15-9bf9-5cc5e272ad12,NEU,WCE,MED
/dat

CORDEX.output.EUR-11.SMHI.MPI-M-MPI-ESM-LR.historical.r2i1p1,SMHI-RCA4,v1,day,sfcWind,version_no,e80b818d-2454-4139-9171-861e7446500b,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.HADGEMr1-COSMOcrCLIM.ref.nc
CORDEX.output.EUR-11.CLMcom-ETH.MOHC-HadGEM2-ES.historical.r1i1p1,CLMcom-ETH-COSMO-crCLIM-v1-1,v1,day,sfcWind,version_no,2c002b76-7459-11ea-8708-d09466354597,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.MPIr1-ALADIN63.ref.nc
CORDEX.output.EUR-11.CNRM.MPI-M-MPI-ESM-LR.historical.r1i1p1,CNRM-ALADIN63,v1,day,sfcWind,version_no,hdl:21.14103/c886f851-d739-4680-92a1-39f12a37716d,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.MIROCr1-WRF361H.ref.nc
CORDEX.output.EUR-11.UHOH.MIROC-MIROC5.historical.r1i1p1,UHOH-WRF361H,v1,day,sfcWind,version_no,80a0cb32-79f4-4348-a3f0-84daf376d61a,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.ECEARTHr12-RACMO.ref.nc
CORDEX.output.EUR-11.KNMI.ICHEC-EC-EARTH.historical.r12i1p1,KNMI-RACMO22E,v1,day,sfcWind,version_no,5807f72b-2015-4ebf-b3be-4f2

CORDEX.output.EUR-11.KNMI.ICHEC-EC-EARTH.rcp85.r1i1p1,KNMI-RACMO22E,v1,day,sfcWind,version_no,556fbc4c-182d-44ec-b336-20b1744fc283,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.NORESMr1-WRF381P.ece.nc
CORDEX.output.EUR-11.IPSL.NCC-NorESM1-M.rcp85.r1i1p1,IPSL-WRF381P,v1,day,sfcWind,version_no,hdl:21.14103/f4c95d93-8115-405c-8c12-72b4dc2be8fc,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.ECEARTHr12-COSMOcrCLIM.ece.nc
CORDEX.output.EUR-11.CLMcom-ETH.ICHEC-EC-EARTH.rcp85.r12i1p1,CLMcom-ETH-COSMO-crCLIM-v1-1,v1,day,sfcWind,version_no,778f5af8-01ef-11ea-a6b9-d094663067f9,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.HADGEMr1-CCLM.ece.nc
CORDEX.output.EUR-11.CLMcom.MOHC-HadGEM2-ES.rcp85.r1i1p1,CLMcom-CCLM4-8-17,v1,day,sfcWind,version_no,0f8f8dc1-0c6f-4fe4-995f-67d54ce702ad,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.IPSLr1-RCA.ece.nc
CORDEX.output.EUR-11.SMHI.IPSL-IPSL-CM5A-MR.rcp85.r1i1p1,SMHI-RCA4,v1,day,sfcWind,version_no,db67d3f8-f04b-43b1-bbd1-a70603e2c1b1,NEU,WCE,MED
/

CORDEX.output.EUR-11.SMHI.MPI-M-MPI-ESM-LR.rcp85.r1i1p1,SMHI-RCA4,v1a,day,sfcWind,version_no,c8d556da-c572-4720-a0b5-c4de400c1644,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.CNRMr1-ALADIN63.ece.nc
CORDEX.output.EUR-11.CNRM.CNRM-CERFACS-CNRM-CM5.rcp85.r1i1p1,CNRM-ALADIN63,v2,day,sfcWind,version_no,hdl:21.14103/158ad7e4-c3c2-4bad-986a-0abfbc4c4c2c,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.MPIr3-RCA.ece.nc
CORDEX.output.EUR-11.SMHI.MPI-M-MPI-ESM-LR.rcp85.r3i1p1,SMHI-RCA4,v1,day,sfcWind,version_no,58e390df-fd26-436b-91de-4c6daeffe46d,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.MIROCr1-REMO.ece.nc
CORDEX.output.EUR-11.GERICS.MIROC-MIROC5.rcp85.r1i1p1,GERICS-REMO2015,v1,day,sfcWind,version_no,f8a8344a-d22c-45b5-bf5c-cf57113109d9,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP85/sfcWind.MPIr1-CCLM.ece.nc
CORDEX.output.EUR-11.CLMcom.MPI-M-MPI-ESM-LR.rcp85.r1i1p1,CLMcom-CCLM4-8-17,v1,day,sfcWind,version_no,17364364-4a50-4a64-86e0-355aba07e5d7,NEU,WCE,MED
/data/jservon/IPCC/WSPD/RCP8

In [19]:
!ncdump -h /data/jservon/IPCC/WSPD/RCP85/sfcWind.IPSLr1-RCA.ece.nc

netcdf sfcWind.IPSLr1-RCA.ece {
dimensions:
	rlon = 424 ;
	rlat = 412 ;
	time = UNLIMITED ; // (1 currently)
	bnds = 2 ;
variables:
	double lon(rlat, rlon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:_CoordinateAxisType = "Lon" ;
	double lat(rlat, rlon) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:_CoordinateAxisType = "Lat" ;
	double rlon(rlon) ;
		rlon:standard_name = "grid_longitude" ;
		rlon:long_name = "longitude in rotated pole grid" ;
		rlon:units = "degrees" ;
		rlon:axis = "X" ;
	double rlat(rlat) ;
		rlat:standard_name = "grid_latitude" ;
		rlat:long_name = "latitude in rotated pole grid" ;
		rlat:units = "degrees" ;
		rlat:axis = "Y" ;
	int rotated_pole ;
		rotated_pole:grid_mapping_name = "rotated_latitude_longitude" ;
		rotated_pole:grid_north_pole_latitude = 39.25 ;
		rotated_pole:grid_north_pole_longitude = -162. ;
	double height ;
		height:standard

In [22]:
!ncdump -h /data/jservon/climafcache/87/90ac33d81a68f449af51a0e0727fc00118b0b99d9c96d4bf4b07e1.nc

netcdf \90ac33d81a68f449af51a0e0727fc00118b0b99d9c96d4bf4b07e1 {
dimensions:
	time = UNLIMITED ; // (2 currently)
	bnds = 2 ;
	x = 527 ;
	y = 527 ;
	m10 = 1 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1949-12-01 00:00:00" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
	double time_bnds(time, bnds) ;
	double x(x) ;
		x:long_name = "x-coordinate in Cartesian system" ;
		x:units = "m" ;
		x:axis = "X" ;
	double y(y) ;
		y:long_name = "y-coordinate in Cartesian system" ;
		y:units = "m" ;
		y:axis = "Y" ;
	float m10(m10) ;
		m10:standard_name = "height" ;
		m10:long_name = "Height level" ;
		m10:units = "m" ;
		m10:positive = "up" ;
		m10:axis = "Z" ;
	double lat(y, x) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude on Cross Points" ;
		lat:units = "degrees_north" ;
	double lon(y, x) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude on Cross Poin

In [20]:
import xarray as xr
ncfile = '/data/jservon/climafcache/87/90ac33d81a68f449af51a0e0727fc00118b0b99d9c96d4bf4b07e1.nc'
nc = xr.open_dataset(ncfile)
tracking_id = nc.tracking_id


In [27]:
#dataset_descr = 'CORDEX.output.'+CORDEX_domain+'.'+ds_kvp['institute']+'.'+ds_kvp['driving_model']+'.'+ds_kvp['experiment']+'.'+ds_kvp['realization']+','+ds_kvp['model_version']+','+ds_kvp['frequency']+','+ds_kvp['variable']+','+'version_no'+','+tracking_id+','+regions

regions = 'NEU,WCE,MED'
institute_id = nc.institute_id
driving_model_id = nc.driving_model_id
frequency = nc.frequency
model_version = nc.rcm_version_id
realization = nc.driving_model_ensemble_member
experiment = nc.driving_experiment_name
rcm = nc.model_id
variable = 'sfcWind'
CORDEX_domain = nc.CORDEX_domain

dataset_descr = 'CORDEX.output.'+CORDEX_domain+'.'+institute_id+'.'+driving_model_id+'.'+rcm+'.'+experiment+'.'+realization+','+model_version+','+frequency+','+variable+','+'version_no'+','+tracking_id+','+regions
print dataset_descr

CORDEX.output.EUR-11.ICTP.MOHC-HadGEM2-ES.ICTP-RegCM4-6.historical.r1i1p1,v1,day,sfcWind,version_no,7f169cf2-6ada-4ff8-928a-3fdf82c5fece,NEU,WCE,MED


In [5]:
ens_CORDEX[clim_period][mem]

time_average(ccdo(ds('wind_individual_models_cordex_ch12%%wind%2029-2048%global%SAM%rcp85%ICHEC-EC-EARTH_r12i1p1_SMHI-RCA4_v3'),operator='yearsum'))